In [ ]:
import torch
import torch.optim as optim
from torch.distributions import MultivariateNormal

In [ ]:
def get_dominant_eigenvalue_torch(matrix):
    # Calculate all eigenvalues
    eigenvalues = torch.linalg.eigvals(matrix)
    
    # Find the eigenvalue with the largest magnitude
    dominant_idx = torch.argmax(torch.abs(eigenvalues))
    dominant_eigenvalue = eigenvalues[dominant_idx]
    
    return dominant_eigenvalue
class Policy(torch.nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc_mean = torch.nn.Linear(dim, dim)
        self.log_std = torch.nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        mean = self.fc_mean(x)
        cov = torch.diag(torch.exp(self.log_std))
        return MultivariateNormal(mean, cov)

def train(A, lambda_, policy, epochs=500):

    optimizer = optim.Adam(policy.parameters())
    v = torch.randn(A.shape[0])
    v = v / torch.norm(v)

    for epoch in range(epochs):
        dist = policy(v)
        delta_v = dist.sample()

        with torch.no_grad():
            v_new = v + delta_v
            v_new = v_new / torch.norm(v_new)
            residual = torch.linalg.norm((A - lambda_ * torch.eye(A.shape[0])) @ v_new)
            reward = -residual.item()

        loss = -dist.log_prob(delta_v).sum() * reward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        v = v_new
        if epoch % 100 == 1:
            print(f'Epoch {_}: Loss = {loss.item()}, Residual = {residual.item()}')
    return v



In [ ]:
get_dominant_eigenvalue_torch(A)

tensor(3.+0.j)

In [ ]:
def evaluate_model(A eigenvector, policy):
    eigenvalue = get_dominant_eigenvalue_torch(A)
    v = train(A, eigenvalue, policy, epochs=1000)
    cosine_similarity = torch.nn.functional.cosine_similarity(v, eigenvector, dim=0)
    print(f'Cosine similarity between v and eigenvector: {cosine_similarity.item()}')
    

In [ ]:
evaluate_model(A, torch.tensor([1.0, -1.0], dtype=torch.float32), policy)

Epoch tensor([-0.7968,  0.6043]): Loss = -1.0955510139465332, Residual = 0.6147363781929016
Epoch tensor([-0.7968,  0.6043]): Loss = -0.40097740292549133, Residual = 0.2780706584453583
Epoch tensor([-0.7968,  0.6043]): Loss = -0.30468595027923584, Residual = 0.2221154123544693
Epoch tensor([-0.7968,  0.6043]): Loss = -0.23056451976299286, Residual = 0.23193545639514923
Epoch tensor([-0.7968,  0.6043]): Loss = -0.044086914509534836, Residual = 0.07378420978784561
Epoch tensor([-0.7968,  0.6043]): Loss = -0.6172485947608948, Residual = 0.3463563919067383
Epoch tensor([-0.7968,  0.6043]): Loss = -0.1758681684732437, Residual = 0.10400097072124481
Epoch tensor([-0.7968,  0.6043]): Loss = -0.5470619797706604, Residual = 0.2510744333267212
Epoch tensor([-0.7968,  0.6043]): Loss = -0.13938060402870178, Residual = 0.12416309863328934
Epoch tensor([-0.7968,  0.6043]): Loss = -0.033290933817625046, Residual = 0.04250981658697128
Cosine similarity between v and eigenvector: 0.9885687828063965
